In [1]:
import pandas as pd
import folium
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import psycopg2
from geopy.distance import geodesic
import os

In [2]:
df_distancia = pd.read_csv('results/dist_estacoes.csv')
estacoes_df = pd.read_csv('base/estacoes.csv')

# Criação do GeoDataFrame a partir das coordenadas
gdf_estacoes = gpd.GeoDataFrame(estacoes_df, geometry=gpd.points_from_xy(estacoes_df.longitude, estacoes_df.latitude))

distancia_filter = 40
nome_variavel = 'velocidade_vento'
variavel_banco = 'ven_vel'

In [3]:
# df_filtrado = df_distancia[df_distancia['distancia_km'] <= distancia_filter]
df_filtrado =  df_distancia[(df_distancia['distancia_km'] <= 40.0) & (df_distancia['estado_origem'].isin(['MG']))]

# Combinar as variáveis em uma única lista e remover duplicados
codigos_estacoes_unicos = set(df_filtrado['cdg_estacao_origem'].tolist() + df_filtrado['cdg_estacao_destino'].tolist())
codigos_estacoes_unicos = list(codigos_estacoes_unicos)

gdf_estacoes = gdf_estacoes[gdf_estacoes['cdg_estacao'].isin(codigos_estacoes_unicos)]
gdf_estacoes



,cidade,situacao,latitude,longitude,altitude,cdg_estacao,sg_estado,geometry
271,JUIZ DE FORA,Operante,-21.769965,-43.364329,-43.364329,A518,MG,POINT (-43.36433 -21.76996)
273,CONCEICAO DAS ALAGOAS,Operante,-19.985860,-48.151574,-48.151574,A520,MG,POINT (-48.15157 -19.98586)
274,BELO HORIZONTE (PAMPULHA),Operante,-19.883889,-43.969444,-43.969444,A521,MG,POINT (-43.96944 -19.88389)
282,PASSA QUATRO,Operante,-22.395833,-44.961944,-44.961944,A529,MG,POINT (-44.96194 -22.39583)
308,IBIRITE (ROLA MOCA),Operante,-20.031389,-44.011111,-44.011111,A555,MG,POINT (-44.01111 -20.03139)
310,CORONEL PACHECO,Operante,-21.546667,-43.261111,-43.261111,A557,MG,POINT (-43.26111 -21.54667)
320,UBERABA,Operante,-19.710000,-47.961944,-47.961944,A568,MG,POINT (-47.96194 -19.71000)
346,TRES RIOS,Operante,-22.098333,-43.208333,-43.208333,A625,RJ,POINT (-43.20833 -22.09833)
426,CACHOEIRA PAULISTA,Operante,-22.688889,-45.005556,-45.005556,A769,SP,POINT (-45.00556 -22.68889)
566,BELO HORIZONTE - CERCADINHO,Operante,-19.980000,-43.958611,-43.958611,F501,MG,POINT (-43.95861 -19.98000)


In [4]:
df_filtrado

,cdg_estacao_origem,cidade_origem,estado_origem,cdg_estacao_destino,distancia_km,diferenca_alt
0,A555,IBIRITE (ROLA MOCA),MG,F501,7.908602,0.052500
1,F501,BELO HORIZONTE - CERCADINHO,MG,A555,7.908602,0.052500
4,A521,BELO HORIZONTE (PAMPULHA),MG,F501,10.700108,0.010833
5,F501,BELO HORIZONTE - CERCADINHO,MG,A521,10.700108,0.010833
14,A555,IBIRITE (ROLA MOCA),MG,A521,16.901248,0.041667
15,A521,BELO HORIZONTE (PAMPULHA),MG,A555,16.901248,0.041667
34,A518,JUIZ DE FORA,MG,A557,26.934424,0.103218
35,A557,CORONEL PACHECO,MG,A518,26.934424,0.103218
63,A529,PASSA QUATRO,MG,A769,32.760884,0.043611
90,A520,CONCEICAO DAS ALAGOAS,MG,A568,36.429878,0.189630


In [5]:
# Criação do mapa centrado nas coordenadas médias das estações
mapa = folium.Map(location=[gdf_estacoes.geometry.y.mean(), gdf_estacoes.geometry.x.mean()], zoom_start=5)

# Loop para adicionar um círculo de 50 km e um marcador para cada estação
for idx, estacao in gdf_estacoes.iterrows():
    
    tooltip_text = f"<b>CDG:</b> {estacao['cdg_estacao']}<br><b>STATUS:</b> {estacao['situacao']}<br><b>CIDADE:</b> {estacao['cidade']}<br><b>ESTADO:</b> {estacao['sg_estado']}"

    folium.Circle(
        location=[estacao['latitude'], estacao['longitude']],
        radius=50000,  # 50 km em metros
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        weight=0.5,  # Espessura da linha do círculo
    ).add_to(mapa)
    
    folium.CircleMarker(
        location=[estacao['latitude'], estacao['longitude']],
        radius=1,  # Tamanho do marcador
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=1,
        tooltip=tooltip_text  # Repetindo o tooltip para o marcador
    ).add_to(mapa)

# Mostra o mapa
mapa


In [6]:
# Parâmetros de conexão
host = "weather-do-user-15558803-0.c.db.ondigitalocean.com"
port = "25060"  # Exemplo: "5432" para PostgreSQL
user = "doadmin"
password = "AVNS_Kru50j5fsOWnNuvMMiQ"
dbname = "weather"
# String de conexão
conn_string = f"host={host} port={port} user={user} password={password} dbname={dbname}"

In [7]:
# Converter a lista de códigos em uma string formatada para o SQL
codigos_formatados = ', '.join(f"'{codigo}'" for codigo in codigos_estacoes_unicos)


with psycopg2.connect(f"host={host} port={port} user={user} password={password} dbname={dbname}") as conn:
        query = f"""
        select hr_medicao, inmet_weather_station_id as cdg_stacao, {variavel_banco} as {nome_variavel}
        from inmet_weather_data
        where inmet_weather_station_id in ({codigos_formatados}) AND
        dta_medicao BETWEEN '2020-01-01' AND '2020-12-31' AND
        {variavel_banco} is not null;
        """

In [8]:
print("Fazendo consulta ")
conn = psycopg2.connect(conn_string)
df_consulta = pd.read_sql_query(query, conn)
conn.close()

Fazendo consulta 


/var/folders/16/b9s1nsbs70sfcvvsld2wjgvm0000gn/T/ipykernel_19787/2816357536.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_consulta = pd.read_sql_query(query, conn)


In [9]:
df = df_consulta.copy()


In [10]:
df

,hr_medicao,cdg_stacao,velocidade_vento
0,2020-02-03 16:00:00,A518,2.0
1,2020-01-01 00:00:00,A518,3.7
2,2020-01-01 01:00:00,A518,3.6
3,2020-01-01 02:00:00,A518,2.9
4,2020-01-01 03:00:00,A518,1.8
...,...,...,...
75263,2020-02-05 00:00:00,A518,4.4
75264,2020-02-05 01:00:00,A518,5.9
75265,2020-02-05 02:00:00,A518,1.2
75266,2020-02-05 03:00:00,A518,1.4


In [11]:
# Converter 'hr_medicao' para datetime e definir como índice do DataFrame, se aplicável
if 'hr_medicao' in df.columns:
    df['hr_medicao'] = pd.to_datetime(df['hr_medicao'])
    df_pivoted = df.pivot_table(index='hr_medicao', columns='cdg_stacao', values=nome_variavel, aggfunc='mean')
    df_pivoted_not_null = df_pivoted.dropna()
    
# Aqui você precisaria calcular df_correlacao para cada tipo de dado
df_correlacao = df_pivoted_not_null.corr()
# Suponha que df_correlacao já está calculado para este exemplo
# Código para construir 'anotacoes' e criar o heatmap, assegurando que 'anotacoes' esteja definido

In [12]:
df_correlacao

cdg_stacao,A518,A520,A521,A529,A555,A557,A568,A769,F501
cdg_stacao,,,,,,,,,
A518,1.000000,0.051203,0.107310,0.233900,0.042304,0.201722,0.023573,0.134563,0.068931
A520,0.051203,1.000000,0.155039,0.244108,-0.046156,0.254835,0.308962,0.302313,-0.118617
A521,0.107310,0.155039,1.000000,0.178460,0.477683,0.221717,0.292559,0.318891,0.388283
A529,0.233900,0.244108,0.178460,1.000000,-0.038322,0.320148,0.113073,0.353587,-0.091795
A555,0.042304,-0.046156,0.477683,-0.038322,1.000000,-0.098553,0.174380,0.021875,0.804647
A557,0.201722,0.254835,0.221717,0.320148,-0.098553,1.000000,0.166025,0.321287,-0.177806
A568,0.023573,0.308962,0.292559,0.113073,0.174380,0.166025,1.000000,0.275692,0.032701
A769,0.134563,0.302313,0.318891,0.353587,0.021875,0.321287,0.275692,1.000000,-0.062038
F501,0.068931,-0.118617,0.388283,-0.091795,0.804647,-0.177806,0.032701,-0.062038,1.000000


In [13]:
# Cria um DataFrame para armazenar os textos de anotação que incluirão correlação e distância
anotacoes = pd.DataFrame(index=df_correlacao.index, columns=df_correlacao.columns, dtype='object')

for origem in df_correlacao.index:
    for destino in df_correlacao.columns:
        correlacao = df_correlacao.loc[origem, destino]
        # Buscar a distância correspondente
        distancia = df_distancia[(df_distancia['cdg_estacao_origem'] == origem) & (df_distancia['cdg_estacao_destino'] == destino)]['distancia_km'].values
            
        if distancia.size > 0:  # Se encontrou a distância
            texto_anotacao = f"{correlacao:.2f}\n{distancia[0]:.2f}km"
        else:
            texto_anotacao = f"{correlacao:.2f}\nN/A"
        anotacoes.loc[origem, destino] = texto_anotacao
    
    # Note que o título e o arquivo de saída devem refletir o tipo de dado sendo processado
plt.figure(figsize=(10, 8))
sns.heatmap(df_correlacao, annot=anotacoes.values, fmt="", cmap='coolwarm', vmin=-1, vmax=1)
plt.title(f'Correlação {nome_variavel.capitalize()} entre estações')
plt.savefig(f'./results/img/estacoes_correlacao/extras/Correção_{nome_variavel}_{distancia_filter}.png')
plt.close()  # Fecha a figura atual para que a próxima possa ser criada sem interferência